<a href="https://colab.research.google.com/github/kinrz/RAFT-Motion-Blur/blob/main/RAFT_Motion_Blur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. Cek GPU & Install Tools
import torch
import subprocess
import os

print("⚙️ Memeriksa System...")
if torch.cuda.is_available():
    print(f"✅ GPU Aktif: {torch.cuda.get_device_name(0)}")
else:
    print("❌ ERROR: GPU tidak terdeteksi. Mohon ganti Runtime Type ke T4 GPU.")
subprocess.run(["apt-get", "install", "-y", "ffmpeg"], stdout=subprocess.DEVNULL)
for f in ['input_videos', 'output_videos', 'temp_frames', 'blur_frames']:
    if os.path.exists(f):
        import shutil
        shutil.rmtree(f)
    os.makedirs(f)
print("✅ System Siap!")

In [ ]:
# @title 2. Upload Video
from google.colab import files
import shutil
import os

uploaded = files.upload()
for filename in uploaded.keys():
    target_path = "input_videos/input.mp4"
    shutil.move(filename, target_path)
    print(f"✅ Video berhasil diupload: {filename} menjadi input.mp4")
    break

In [ ]:
# @title 3. Settings
import os
import torch
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torchvision.models.optical_flow import raft_large, Raft_Large_Weights
import cv2
import numpy as np
from tqdm.notebook import tqdm
import subprocess
import shutil
from glob import glob
import gc

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# @markdown **Blur Direction:**
# @markdown * `Forward`: Blur dihitung dari frame setelahnya.
# @markdown * `Backward`: Blur dihitung dari frame sebelumnya.
Blur_Direction = "Backward" #@param ["Forward", "Backward"]
# @markdown **Tail Expansion:**
# @markdown Feather tepi motion blur. Atur sesuai selera.
# @markdown * `0`: Off (Tepi tajam).
# @markdown * `30`: Tepi blur halus **Recommended**.
Tail_Expansion = 40 #@param {type:"slider", min:0, max:100, step:5}
# @markdown **Blur Strength:**
# @markdown Panjang Blur
# @markdown * `0.8`: Jika bingung mau atur berapa.
Blur_Strength = 1 #@param {type:"slider", min:0.1, max:3.0, step:0.1}
# @markdown **Blur Sample:**
# @markdown Singkatnya kualitas blur. Jika diatur rendah blur akan berbayang.
# @markdown * `128`: Atur ke tertinggi.
Num_Samples = 128 #@param {type:"slider", min:16, max:128, step:8}
# @markdown **Flow Iterations:**
# @markdown Berapa kali setiap frame dibaca engine. Semakin tinggi semakin detail objek kecil dibaca. Dapat mempengaruhi kecepatan rendering.
Flow_Iterations = 16 #@param {type:"slider", min:10, max:40, step:2}
# @markdown **VRAM Safe Mode:**
# @markdown Nyalakan jika sering gagal render. Kebanyakan kasus di resolusi tinggi
VRAM_Safe_Mode = False #@param {type:"boolean"}
# @markdown **Safe Mode Limit (jika VRAM safe nyala):**
# @markdown Batas dimensi tertinggi video untuk dibaca engine. Resolusi video masih tetap asli.
Safe_Resolution = 1280 #@param {type:"slider", min:960, max:1280, step:64}
# @markdown **Consistency Check (Eksperimental):**
# @markdown Untuk kasus jika video sudah terdapat objek screen space seperti Black bar, watermark, Overlay dll yang terus nempel dilayar.
# @markdown * Tidak disarankan
Consistency_Check = False #@param {type:"boolean"}
# @markdown **Video Quality (CRF)**
Video_Quality = 19 #@param {type:"slider", min:15, max:30, step:1}
Internal_Resolution_Limit = Safe_Resolution if VRAM_Safe_Mode else 0
print(f"Getting RAFT engine...")
print(f"► Direction: {Blur_Direction}")
print(f"► Expansion: {Tail_Expansion}")
print(f"► Blur Strength: {Blur_Strength}")
print(f"► Iterations: {Flow_Iterations}")
print(f"► VRAM Safe: {'ON (' + str(Internal_Resolution_Limit) + 'px)' if VRAM_Safe_Mode else 'OFF (Native)'}")
print(f"► Blur Samples: {Num_Samples}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
weights = Raft_Large_Weights.DEFAULT
transforms = weights.transforms()
model = raft_large(weights=weights, progress=False).to(device)
model.eval()
def get_grid(H, W):
    y = torch.linspace(-1, 1, H, device=device)
    x = torch.linspace(-1, 1, W, device=device)
    gy, gx = torch.meshgrid(y, x, indexing='ij')
    return torch.stack((gx, gy), dim=2).unsqueeze(0)
def calculate_consistency(flow_fwd, flow_bwd):
    N, C, H, W = flow_fwd.shape
    base_grid = get_grid(H, W)
    flow_fwd_norm = flow_fwd.permute(0, 2, 3, 1).clone()
    flow_fwd_norm[..., 0] /= (W / 2.0)
    flow_fwd_norm[..., 1] /= (H / 2.0)
    grid = base_grid + flow_fwd_norm
    warped_bwd = F.grid_sample(flow_bwd, grid, mode='bilinear', padding_mode='reflection', align_corners=True)
    diff = flow_fwd + warped_bwd
    magnitude = torch.norm(diff, dim=1, keepdim=True)
    return torch.exp(-0.5 * magnitude)
def apply_vector_blur_complex(img_tensor, flow_tensor, samples, strength, expansion, mask=None):
    N, C, H, W = img_tensor.shape
    base_grid = get_grid(H, W)
    accumulated_img = torch.zeros_like(img_tensor)
    processed_flow = flow_tensor.clone()
    if expansion > 0:
        k_size = int(expansion) * 2 + 1
        processed_flow = TF.gaussian_blur(processed_flow, kernel_size=k_size, sigma=expansion)
    flow_norm = processed_flow.permute(0, 2, 3, 1).clone()
    flow_norm[..., 0] /= (W / 2.0)
    flow_norm[..., 1] /= (H / 2.0)
    for i in range(samples):
        t = (i / (samples - 1)) - 0.5
        offset = flow_norm * (t * strength)
        sampling_grid = base_grid + offset
        warped = F.grid_sample(img_tensor, sampling_grid, mode='bilinear', padding_mode='reflection', align_corners=True)
        accumulated_img += warped
    blurred = accumulated_img / samples
    if mask is not None:
        return (blurred * mask) + (img_tensor * (1 - mask))
    else:
        return blurred
def pad_to_8(tensor):
    h, w = tensor.shape[-2:]
    new_h = ((h + 7) // 8) * 8
    new_w = ((w + 7) // 8) * 8
    ph = new_h - h
    pw = new_w - w
    if ph > 0 or pw > 0:
        return F.pad(tensor, (0, pw, 0, ph)), h, w
    return tensor, h, w
input_video = "input_videos/input.mp4"
output_video = "output_videos/result.mp4"
temp_dir = "temp_frames"
blur_dir = "blur_frames"
print("Extracting Frames...")
subprocess.run(f"ffmpeg -i {input_video} -pix_fmt rgb24 {temp_dir}/%08d.png -y -hide_banner -loglevel error", shell=True)
frame_files = sorted(glob(f"{temp_dir}/*.png"))
print(f"Rendering Progress...")
for i in tqdm(range(len(frame_files)), desc="Applying Motion Blur"):
    curr_path = frame_files[i]
    if Blur_Direction == "Forward":
        if i == len(frame_files) - 1:
            shutil.copy(curr_path, f"{blur_dir}/{i:08d}.png")
            continue
        target_path = frame_files[i+1]
    else:
        if i == 0:
            shutil.copy(curr_path, f"{blur_dir}/{i:08d}.png")
            continue
        target_path = frame_files[i-1]
    img1_orig = cv2.cvtColor(cv2.imread(curr_path), cv2.COLOR_BGR2RGB)
    img2_orig = cv2.cvtColor(cv2.imread(target_path), cv2.COLOR_BGR2RGB)
    orig_h, orig_w = img1_orig.shape[:2]
    calc_h, calc_w = orig_h, orig_w
    scale_factor = 1.0
    if Internal_Resolution_Limit > 0 and (orig_w > Internal_Resolution_Limit or orig_h > Internal_Resolution_Limit):
        if orig_w >= orig_h: scale_factor = Internal_Resolution_Limit / orig_w
        else: scale_factor = Internal_Resolution_Limit / orig_h
        calc_w, calc_h = int(orig_w * scale_factor), int(orig_h * scale_factor)
        img1_small = cv2.resize(img1_orig, (calc_w, calc_h), interpolation=cv2.INTER_LINEAR)
        img2_small = cv2.resize(img2_orig, (calc_w, calc_h), interpolation=cv2.INTER_LINEAR)
    else:
        img1_small, img2_small = img1_orig, img2_orig
    img1_t_full = TF.to_tensor(img1_orig).unsqueeze(0).to(device)
    img1_t_calc = TF.to_tensor(img1_small).unsqueeze(0).to(device)
    img2_t_calc = TF.to_tensor(img2_small).unsqueeze(0).to(device)
    img1_batch, img2_batch = transforms(img1_t_calc, img2_t_calc)
    img1_pad, pad_h, pad_w = pad_to_8(img1_batch)
    img2_pad, _, _ = pad_to_8(img2_batch)
    with torch.no_grad():
        flow_main = model(img1_pad, img2_pad, num_flow_updates=Flow_Iterations)[-1]
        consistency_mask = None
        if Consistency_Check:
            flow_reverse = model(img2_pad, img1_pad, num_flow_updates=Flow_Iterations)[-1]
            mask_pad = calculate_consistency(flow_main, flow_reverse)
            consistency_mask = mask_pad[:, :, :pad_h, :pad_w]
        del img1_batch, img2_batch, img1_pad, img2_pad
        torch.cuda.empty_cache()
    flow_small = flow_main[:, :, :pad_h, :pad_w]
    if scale_factor != 1.0:
        flow_final = F.interpolate(flow_small, size=(orig_h, orig_w), mode='bilinear', align_corners=False)
        flow_final *= (1.0 / scale_factor)
        if consistency_mask is not None:
            consistency_mask = F.interpolate(consistency_mask, size=(orig_h, orig_w), mode='bilinear', align_corners=False)
    else:
        flow_final = flow_small
    blurred_tensor = apply_vector_blur_complex(
        img1_t_full, flow_final, Num_Samples, Blur_Strength, Tail_Expansion, consistency_mask
    )
    res_img = (blurred_tensor.squeeze(0).permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
    res_img = cv2.cvtColor(res_img, cv2.COLOR_RGB2BGR)
    cv2.imwrite(f"{blur_dir}/{i:08d}.png", res_img)
    del flow_main, flow_small, flow_final, blurred_tensor, img1_t_full, img1_t_calc, img2_t_calc
    if consistency_mask is not None: del consistency_mask
    torch.cuda.empty_cache()
print("Encoding Final Video...")
fps_cmd = "ffmpeg -i input_videos/input.mp4 2>&1 | sed -n 's/.*, \\(.*\\) fp.*/\\1/p'"
orig_fps = subprocess.check_output(fps_cmd, shell=True).decode("utf-8").strip()
if not orig_fps: orig_fps = 30
subprocess.run(f"ffmpeg -r {orig_fps} -i {blur_dir}/%08d.png -i {input_video} -map 0:v -map 1:a -c:a copy -c:v h264_nvenc -profile:v high -level 4.1 -preset p6 -tune hq -rc constqp -qp {Video_Quality} -pix_fmt yuv420p {output_video} -y -hide_banner -loglevel error", shell=True)
print(f"✅ SELESAI! Hasil tersimpan di: {output_video}")

In [ ]:
# @title 4. Download Video
from google.colab import files
files.download(output_video)